### Project

For this project you have 4 files containing information about persons.

The files are:
* `personal_info.csv` -   personal information such as name, gender, etc. (one row per person)
* `vehicles.csv` -   what vehicle people own (one row per person)
* `employment.csv` -   where a person is employed (one row per person)
* `update_status.csv` -   when the person's data was created and last updated

Each file contains a key, `SSN`, which **uniquely** identifies a person.

This key is present in **all** four files.

You are guaranteed that the same SSN value is present in **every** file, and that it only appears **once per file**.

In addition, the files are all sorted by SSN, i.e. the SSN values appear in the same order in each file.

##### Goal 1

Your first task is to create iterators for each of the four files that contained cleaned up data, of the correct type (e.g. string, int, date, etc), and represented by a named tuple.

For now these four iterators are just separate, independent iterators.

##### Goal 2

Create a single iterable that combines all the columns from all the iterators.

The iterable should yield named tuples containing all the columns.
Make sure that the SSN's across the files match!

All the files are guaranteed to be in SSN sort order, and every SSN is unique, and every SSN appears in every file.

Make sure the SSN is not repeated 4 times - one time per row is enough!

##### Goal 3

Next, you want to identify any stale records, where stale simply means the record has not been updated since 3/1/2017 (e.g. last update date < 3/1/2017). Create an iterator that only contains current records (i.e. not stale) based on the `last_updated` field from the `status_update` file.

##### Goal 4

Find the largest group of car makes for each gender.

Possibly more than one such group per gender exists (equal sizes).

#### Hints

You will not be able to use a simple split approach here, as I explain in the video.

Instead you should use the `csv` module and the `reader` function.

Here's a simple example of how to use it - you will need to expand on this for your project goals, but this is a good starting point.

In [1]:
import csv

def read_file(file_name):
    with open(file_name) as f:
        rows = csv.reader(f, delimiter=',', quotechar='"')
        yield from rows
    

In [2]:
from itertools import islice

rows = read_file('personal_info.csv')
for row in islice(rows, 5):
    print(row)

['ssn', 'first_name', 'last_name', 'gender', 'language']
['100-53-9824', 'Sebastiano', 'Tester', 'Male', 'Icelandic']
['101-71-4702', 'Cayla', 'MacDonagh', 'Female', 'Lao']
['101-84-0356', 'Nomi', 'Lipprose', 'Female', 'Yiddish']
['104-22-0928', 'Justinian', 'Kunzelmann', 'Male', 'Dhivehi']


As you can see, the data is already separated into a list containing the individual fields - but of course they are all just strings.

### Good luck!

In [3]:
from collections import namedtuple, Counter
from itertools import groupby
from datetime import datetime, timezone
import csv

In [4]:
fname_personal = 'personal_info.csv'
fname_employment = 'employment.csv'
fname_vehicles = 'vehicles.csv'
fname_update_status = 'update_status.csv'
fnames = {'personal_info':fname_personal, 'employment':fname_employment, 'vehicles':fname_vehicles, 'update_status':fname_update_status}

### Goal 1

In [5]:
def determine_dtype(string):
    try:
        int(string)
        return int
    except ValueError:
        try:
            float(string)
            return float
        except ValueError:
            try:
                datetime.strptime(string, '%Y-%m-%dT%H:%M:%S%z')
                return lambda x: datetime.strptime(x, '%Y-%m-%dT%H:%M:%S%z')
            except ValueError:
                return str

def safe_parse(parser, val_str):
    if not val_str:
        return None
    try:
        return parser(val_str)
    except ValueError:
        return None

In [6]:
def parsed_data(file_name):
    with open(file_name, newline='') as csvfile:
        csvreader = csv.reader(csvfile)
        fieldnames = ['_'.join(fieldname.lower().split()) for fieldname in next(csvreader)]
        Record = namedtuple('Record', fieldnames, rename=True)
        first_row_values_str = next(csvreader)
        dtype_parsers = [determine_dtype(string) for string in first_row_values_str]
        first_row_parsed = Record(*[safe_parse(parser, val_str) for val_str, parser in zip(first_row_values_str, dtype_parsers)])
        yield first_row_parsed
        for row in csvreader:
            yield Record(*[safe_parse(parser, val_str) for val_str, parser in zip(row, dtype_parsers)])

In [7]:
file_iters = {table: parsed_data(fname) for table, fname in fnames.items()}

### Goal 2

In [8]:
def iter_combined(fnames):
    iters_zip = zip(*[parsed_data(fname) for fname in fnames.values()])
    merged_record_dict = {k:v for record in next(iters_zip) for k, v in record._asdict().items()}
    MergeRecord = namedtuple('MergeRecord', merged_record_dict.keys())
    yield MergeRecord(**merged_record_dict)
    for records in iters_zip:
        yield MergeRecord(**{k:v for record in records for k, v in record._asdict().items()})

In [9]:
data_iter = iter_combined(fnames)

### Goal 3

In [10]:
cutoff_date = datetime(2017, 3, 1,tzinfo=timezone.utc)

In [11]:
def filtered_iter_combined(fnames, *, key=lambda x: x.last_updated >= cutoff_date):
    yield from filter(key, iter_combined(fnames))

In [12]:
updated_records = [*filtered_iter_combined(fnames)]

### Goal 4

In [13]:
for key, group in groupby(sorted(filtered_iter_combined(fnames), key=lambda x: x.gender), lambda x: x.gender):
    value_counts = Counter(r.vehicle_make for r in group).most_common()
    max_count = value_counts[0][1]
    print(key)
    print([(make, count) for make, count in value_counts if count == max_count])

Female
[('Chevrolet', 42), ('Ford', 42)]
Male
[('Ford', 40)]


Female
[('Chevrolet', 42), ('Ford', 42)]
Male
[('Ford', 40)]